In [269]:
import cv2

img_bgr = cv2.imread('image2.jpg') #Image is being read in BGR color palette
img = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
print(img.shape)

(480, 640, 3)


In [254]:
#1.1.1 Shadow-Map Generation

#RGB to HSI

import numpy as np

img_np = np.asarray(img)
print(img_np.shape)

h, w, c = img_np.shape

I = np.zeros((h, w, 1))
V1 = np.zeros((h, w, 1))
V2 = np.zeros((h, w, 1))
S = np.zeros((h, w, 1))
H = np.zeros((h, w, 1))

print(img[0][0])

hsi_mat = np.array([[1/3, 1/3, 1/3], [-(np.sqrt(6))/6, -(np.sqrt(6))/6, (np.sqrt(6))/3], [(1/np.sqrt(6)), -(2/np.sqrt(6)), 0]]) #Eqn 1
print(hsi_mat)

for y in range(h):
    for x in range(w):
        rgb = img_np[y][x]
        rgb = rgb.reshape((3, 1))
        out = np.matmul(hsi_mat, rgb)
        I[y][x] = out[0][0]
        V1[y][x] = out[1][0]
        V2[y][x] = out[2][0]
        S[y][x] = np.sqrt((np.square(V1[y][x])) + np.square(V2[y][x])) #Eqn 2
        if V1[y][x] != 1:
            if V1[y][x] == 0 and V2[y][x] == 0:
                H[y][x] = 0
                continue
            elif V1[y][x] == 0:
                H[y][x] = 0
                continue
            H[y][x] = np.arctan((V2[y][x])/V1[y][x]) #Eqn 3


(480, 640, 3)
[162 215 255]
[[ 0.33333333  0.33333333  0.33333333]
 [-0.40824829 -0.40824829  0.81649658]
 [ 0.40824829 -0.81649658  0.        ]]


In [255]:
H_norm = (H - np.min(H)) / (np.max(H) - np.min(H))
I_norm = (I - np.min(I)) / (np.max(I) - np.min(I))

In [256]:
#Computing r-map
r_map = (H_norm + 1) / (I_norm + 1) #Eqn 4

r_map_scaled = 255 * ((r_map - np.min(r_map)) / (np.max(r_map) - np.min(r_map)))

In [257]:
#Shadow map and Threshold
P = {}

for y in range(r_map_scaled.shape[0]):
    for x in range(r_map_scaled.shape[1]):
        keys_list = P.keys()
        if int(r_map_scaled[y][x]) in keys_list:
            P[int(r_map_scaled[y][x])] += 1
        else:
            P[int(r_map_scaled[y][x])] = 1

for key in P.keys():
    P[key] = P[key] / (h * w)

T_dict = {}

for t in range(0, 256):
    W1, W2 = 0, 0
    for i in range(0, t+1):
        if i in P.keys():
            W1 += P[i]
    for i in range(t+1, 256):
        if i in P.keys():
            W2 += P[i]

    mu1, mu2 = 0, 0
    for i in range(0, t+1):
        if i in P.keys():
            mu1 += (i * P[i]) / W1
    for i in range(t+1, 256):
        if i in P.keys():
            mu2 += (i * P[i]) / W2

    t1, t2 = 0, 0
    for i in range(0, t+1):
        if i in P.keys():
            t1 += P[i] * ((i - mu1) ** 2)
    for i in range(t+1, 256):
        if i in P.keys():
            t2 += P[i] * ((i - mu2) ** 2)

    T_dict[t] = t1+t2


lt = []
for key in T_dict.keys():
    lt.append(T_dict[key])

T = lt.index(min(lt))

In [258]:
#Shadow map
s = np.where(r_map_scaled > T, 1, 0)

s_inv = np.where(r_map_scaled > T, 0, 1)
cv2.imwrite('shadow_map.jpg', s_inv * 255)

True

In [318]:
#Shadow Image
lmd_1 = 0.7
SI_1 = np.where(s_inv == 0, lmd_1 * img_bgr + (1 - lmd_1) * s_inv, img_bgr)

cv2.imwrite('shadow_image_lmb_0.8.jpg', SI_1)

lmd_2 = 1.3
SI_2 = np.where(s_inv == 0, lmd_2 * img_bgr + (1 - lmd_2) * s_inv, img_bgr)

cv2.imwrite('shadow_image_lmd_1.3.jpg', SI_2)

True

In [260]:
#1.1.2 Line Draft Generation

img_gray = cv2.imread('image2.jpg', 0)

def bilateral_filter(image, diameter, sigma_color, sigma_space):
    filtered_image = np.zeros_like(image)
    height, width = image.shape

    for y in range(height):
        for x in range(width):
            weighted_sum = 0
            normalization = 0

            for i in range(-diameter // 2, diameter // 2 + 1):
                for j in range(-diameter // 2, diameter // 2 + 1):
                    neighbor_y = y + i
                    neighbor_x = x + j

                    if 0 <= neighbor_y < height and 0 <= neighbor_x < width:
                        color_difference = image[neighbor_y, neighbor_x] - image[y, x]
                        color_weight = np.exp(-np.square(color_difference) / (2 * sigma_color * sigma_color))
                        spatial_weight = np.exp(-np.square(i) + np.square(j) / (2 * sigma_space * sigma_space))

                        weight = color_weight * spatial_weight

                        weighted_sum += weight * image[neighbor_y, neighbor_x]
                        normalization += weight

            filtered_image[y, x] = weighted_sum / normalization

    return filtered_image

In [10]:
diameter = 3
sigma_colors = [10, 30, 50, 70, 100]
sigma_spaces = [10, 30, 50, 70, 100]

#Bilateral Filtering
for sigma_color in sigma_colors:
    for sigma_space in sigma_spaces:
        filtered_image = bilateral_filter(img_gray, diameter, sigma_color, sigma_space)
        cv2.imwrite('bilateral_output' + '_' + str(sigma_color) + '_' +str(sigma_space) + '.jpg', filtered_image)

/var/folders/hx/fhnqsgs157jf_ft4qz5_j92c0000gn/T/ipykernel_71052/188239627.py:20: RuntimeWarning: overflow encountered in ubyte_scalars
  color_difference = image[neighbor_y, neighbor_x] - image[y, x]


KeyboardInterrupt: 

In [261]:
#Test block

diameter = 5
sigma_colors = [50,]
sigma_spaces = [50,]

#Bilateral Filtering
for sigma_color in sigma_colors:
    for sigma_space in sigma_spaces:
        filtered_image = bilateral_filter(img_gray, diameter, sigma_color, sigma_space)
        cv2.imwrite('dia_5/bilateral_output' + '_' + str(sigma_color) + '_' +str(sigma_space) + '.jpg', filtered_image)

/var/folders/hx/fhnqsgs157jf_ft4qz5_j92c0000gn/T/ipykernel_6936/188239627.py:20: RuntimeWarning: overflow encountered in ubyte_scalars
  color_difference = image[neighbor_y, neighbor_x] - image[y, x]


In [262]:
#Edge Detection

#Best sigma_color = , Best sigma_space = 
bil_img = cv2.imread('dia_5/bilateral_output_50_50.jpg', 0)

sobel_x = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
sobel_y = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]])

padded_image = np.pad(bil_img, ((1, 1), (1, 1)), mode='edge')

gradient_x = np.zeros_like(bil_img, dtype=np.float32)
gradient_y = np.zeros_like(bil_img, dtype=np.float32)

for y in range(bil_img.shape[0]):
    for x in range(bil_img.shape[1]):
        gradient_x[y, x] = np.sum(padded_image[y:y+3, x:x+3] * sobel_x)
        gradient_y[y, x] = np.sum(padded_image[y:y+3, x:x+3] * sobel_y)

edge_map = np.sqrt(gradient_x**2 + gradient_y**2)

cv2.imwrite('edge_map_dia5_50,50.jpg', edge_map)

True

In [263]:
#Line Draft using Threshold
ld_thresh = 130

line_draft = np.where(edge_map >= ld_thresh, 1, 0)

line_draft_inv = 255 * np.where(edge_map >= ld_thresh, 0, 1)

cv2.imwrite('line_draft_inv_dia5_thr120.jpg', line_draft_inv)

True

In [349]:
#1.2 Color Adjustment Step

#img_bgr_processed = img_bgr.astype("float32") / 255

lab_img = cv2.cvtColor(img_bgr.copy(), cv2.COLOR_BGR2LAB)

l, a, b = np.float64(lab_img[:,:,0]), np.float64(lab_img[:,:,1]), np.float64(lab_img[:,:,2])
# l_mid = np.int8(50)
# l_mid = float(127)
l_mid = 60
print(np.min(l), np.max(l)) 
# l = ((l - np.min(l)) / (np.max(l) - np.min(l))) * 100.0


neutral_l = np.ones((h, w)) * l_mid
# neutral_l = np.array(neutral_l, dtype=np.uint8)

chromatic_map = np.dstack((neutral_l, a, b))
print(chromatic_map.shape)
chromatic_map = np.array(chromatic_map, dtype = np.uint8)

chromatic_map_rgb = cv2.cvtColor(chromatic_map.copy(), cv2.COLOR_LAB2RGB)

cv2.imwrite('lab_img.jpg', lab_img)
cv2.imwrite('chromatic_map.jpg', cv2.cvtColor(chromatic_map_rgb, cv2.COLOR_RGB2BGR))

0.0 255.0
(480, 640, 3)


True

In [358]:
#SI is BGR

SI = SI_1.copy()
rho = 0.005 #[0.005, 0.2]

SI = np.array(SI, dtype=np.uint8)

SI = cv2.cvtColor(SI, cv2.COLOR_BGR2RGB)

SI = np.array(SI, dtype=np.float64)

SI_prime = SI * ((1 + np.tanh(rho * (chromatic_map_rgb - 128))) / 2)
SI_prime = np.array(SI_prime, dtype=np.uint8)

cv2.imwrite('shadow_image_color_enhanced.jpg', cv2.cvtColor(SI_prime, cv2.COLOR_RGB2BGR))

True

In [369]:
#Saturation Correction

saturation_scale = 1.3

# SI_corrected = SI_prime * saturation_scale
# SI_corrected = np.clip(SI_corrected, 0, 255)
# SI_corrected = np.array(SI_corrected, dtype=np.uint8)

SI_prime_HSV = cv2.cvtColor(SI_prime, cv2.COLOR_RGB2HSV)

H = SI_prime_HSV[:, :, 0].copy()
S = SI_prime_HSV[:, :, 1].copy()
V = SI_prime_HSV[:, :, 2].copy()

S_corrected = np.array(np.round(255 * ((S - np.min(S)) / (np.max(S) - np.min(S))))).astype(int)

S_corrected = S_corrected * saturation_scale
S_corrected = np.clip(S_corrected, 0, 255)

S_corrected = np.array(S_corrected, dtype=np.uint8)

SI_corrected = np.stack((H, S_corrected, V), axis = -1) #Why not S

SI_corrected_RGB = cv2.cvtColor(SI_corrected, cv2.COLOR_HSV2RGB)

cv2.imwrite('shadow_image_corrected.jpg', cv2.cvtColor(SI_corrected, cv2.COLOR_HSV2BGR))

True

In [363]:
# #Test of histogram equilization on s channel of HSV image
# SI_prime_HSV = cv2.cvtColor(SI_prime, cv2.COLOR_RGB2HSV)

# H = SI_prime_HSV[:, :, 0].copy()
# S = SI_prime_HSV[:, :, 1].copy()
# V = SI_prime_HSV[:, :, 2].copy()

# def hist_of_image(l):
#     intensity_dict = {}
#     h, w = l.shape
#     for i in range(0, 256):
#         intensity_dict[i] = 0
#     for y in range(h):
#         for x in range(w):
#             intensity_dict[int(l[y, x])] += 1
#     # keys = list(intensity_dict.keys())
#     # keys.sort()
#     # sorted_dict = {i: intensity_dict[i] for i in keys}
#     return intensity_dict

# def hist_eq(src_img, l_dict):
#     h, w = src_img.shape
#     pr = 0
#     pixel_cdf = {}
#     intensity_map = {}

#     keys = list(l_dict.keys())
#     for i in keys:
#         # if not bool(unnorm_cdf):
#         #     unnorm_cdf[i] = l_dict[i]
#         # else:
#         #     unnorm_cdf[i] = unnorm_cdf[int(keys[i-1])] + l_dict[i]
#         pr += l_dict[i] / (h * w)
#         pixel_cdf[i] = pr
#         intensity = int(pr * 255.0)
#         intensity_map[i] = intensity

#     eq_image = src_img.copy()
#     for y in range(h):
#         for x in range(w):
#             eq_image[y, x] = intensity_map[int(eq_image[y, x])]

#     return eq_image

# hist_dictionary = hist_of_image(S.copy())
# S_corrected = hist_eq(S, hist_dictionary)
# S_corrected = np.array(S_corrected, dtype=np.uint8)

# SI_corrected = np.stack((H, S_corrected, V), axis = -1) #Why not S

# SI_corrected_RGB = cv2.cvtColor(SI_corrected, cv2.COLOR_HSV2RGB)

# cv2.imwrite('shadow_image_corrected.jpg', cv2.cvtColor(SI_corrected, cv2.COLOR_HSV2BGR))

True

In [370]:
#Artistic Enhanced Image
beta = 0.75 #[0, 1]

line_draft_inv_3d = np.stack([line_draft_inv, line_draft_inv, line_draft_inv], axis = -1) 

artistic_image = np.where(line_draft_inv_3d == 0, beta * SI_corrected_RGB, SI_corrected_RGB)

artistic_image = np.array(artistic_image, dtype=np.uint8)

cv2.imwrite('artistic_image.jpg', cv2.cvtColor(artistic_image, cv2.COLOR_RGB2BGR))

True

In [16]:
#2 Quantization

#Median Cut

median_cut_image = artistic_image.copy()

def median_cut_quantize(img, flattened_arr):
    r_avg = np.mean(flattened_arr[:,0])
    g_avg = np.mean(flattened_arr[:,1])
    b_avg = np.mean(flattened_arr[:,2])
    
    for data in flattened_arr:
        img[data[3], data[4]] = [r_avg, g_avg, b_avg]

def split_into_bins(img, flattened_arr, depth):  
    if depth == 0:
        median_cut_quantize(img, flattened_arr)
        return  
    ranges = np.ptp(flattened_arr, axis=0)
    r_range, g_range, b_range = ranges[0], ranges[1], ranges[2]

    rgb_val = [r_range, g_range, b_range]
    range_idx = rgb_val.index(max(rgb_val))

    flattened_arr = flattened_arr[flattened_arr[:,range_idx].argsort()]
    median_index = int((len(flattened_arr)+1)/2)

    split_into_bins(img, flattened_arr[0:median_index], depth-1)
    split_into_bins(img, flattened_arr[median_index:], depth-1)


flattened_img_array = []
for rindex, rows in enumerate(median_cut_image):
    for cindex, color in enumerate(rows):
        flattened_img_array.append([color[0],color[1],color[2],rindex, cindex]) 
        
flattened_img_array = np.array(flattened_img_array)
        
split_into_bins(median_cut_image, flattened_img_array, 5) #2^2 = 4 colors

cv2.imwrite('median_cut_quantized_image.jpg', cv2.cvtColor(median_cut_image, cv2.COLOR_RGB2BGR))



True

In [17]:
#Floyed Steinbeirg Dithering

# color_palette = 4 #4 colours per channel of RGB

# rgb_values = np.linspace(0, 255, num=color_palette)
# r_values, g_values, b_values = rgb_values, rgb_values, rgb_values

# r_values = list(set(median_cut_image[:, :, 0].flatten()))
# g_values = list(set(median_cut_image[:, :, 1].flatten()))
# b_values = list(set(median_cut_image[:, :, 2].flatten()))

# print(r_values, g_values, b_values)

# indices = np.indices((len(r_values), len(g_values), len(b_values)))

# combinations = np.array(np.meshgrid(r_values, g_values, b_values)).T.reshape(-1, 3)

In [18]:

# for y in range(h):
#     for x in range(w):
#         pix = median_cut_image[y, x]
        

In [20]:
quantized_image = median_cut_image.copy()

original_image = artistic_image.copy()
# original_image = cv2.imread('image2.jpg', )

flattened_array = median_cut_image.reshape(-1, 3)
unique_pixel_values = np.array(np.unique(flattened_array, axis=0), dtype=np.float64)

def calc_distance(p1, p2):
    return np.sum(np.abs(p1 - p2))

def nearestpixel(pix):
    min_distance = float('inf')
    closest_pixel = None
    
    for unique_pixel in unique_pixel_values:
        distance = calc_distance(pix, unique_pixel)
        if distance < min_distance:
            min_distance = distance
            closest_pixel = unique_pixel
    return closest_pixel

def apply_floyd_steinberg_dithering(org_image, quant_image):
    new_image = np.array(org_image.copy(), dtype=np.float64)
    height, width, _ = new_image.shape

    for y in range(height):
        for x in range(width):
            pix = new_image[y, x].copy()
            new_pixel = nearestpixel(pix.copy())
            new_image[y, x] = new_pixel
            
            quantization_error = pix - new_pixel

            if x + 1 < width:
                new_image[y, x + 1] = new_image[y, x + 1] + quantization_error * 7 / 16
            if y + 1 < height:
                if x - 1 >= 0:
                    new_image[y + 1, x - 1] = new_image[y + 1, x - 1] + quantization_error * 3 / 16
                quant_image[y + 1, x] = quant_image[y + 1, x] + quantization_error * 5 / 16
                if x + 1 < width:
                    new_image[y + 1, x + 1] = new_image[y + 1, x + 1] + quantization_error * 1 / 16
    
    dithered_image = np.array(new_image, dtype=np.uint8)
    return dithered_image

cv2.imwrite('sample1.jpg', cv2.cvtColor(original_image, cv2.COLOR_RGB2BGR))
cv2.imwrite('sample2.jpg', cv2.cvtColor(quantized_image, cv2.COLOR_RGB2BGR))

dithered_image = apply_floyd_steinberg_dithering(original_image, quantized_image)

# Save the dithered image
dithered_image_path = 'dithered_image_quant.jpg'
cv2.imwrite('dithered_image.jpg', cv2.cvtColor(dithered_image, cv2.COLOR_RGB2BGR))


True

In [19]:
#3 Color Transfer

# Histogram Matching

# def hist_of_image(l):
#     intensity_dict = {}
#     h, w = l.shape
#     for y in range(h):
#         for x in range(w):
#             if int(l[y, x]) not in intensity_dict.keys():
#                 intensity_dict[int(l[y, x])] = 1
#             else:
#                 intensity_dict[int(l[y, x])] += 1
#     keys = list(intensity_dict.keys())
#     keys.sort()
#     sorted_dict = {i: intensity_dict[i] for i in keys}
#     return sorted_dict

# def hist_eq(src_img, l_dict):
#     h, w = src_img.shape
#     pr = 0
#     pixel_cdf = {}
#     intensity_map = {}

#     keys = list(l_dict.keys())
#     for i in keys:
#         # if not bool(unnorm_cdf):
#         #     unnorm_cdf[i] = l_dict[i]
#         # else:
#         #     unnorm_cdf[i] = unnorm_cdf[int(keys[i-1])] + l_dict[i]
#         pr += l_dict[i] / (h * w)
#         pixel_cdf[i] = pr
#         intensity = int(pr * 255.0)
#         intensity_map[i] = intensity

#     eq_image = src_img.copy()
#     for y in range(h):
#         for x in range(w):
#             eq_image[y, x] = intensity_map[int(eq_image[y, x])]

#     return eq_image, pixel_cdf

# source_image = artistic_image.copy()
# source_image = cv2.cvtColor(source_image, cv2.COLOR_RGB2LAB)

# source_image_l = source_image[:, :, 0]
# a, b = source_image[:, :, 1], source_image[:, :, 2]

# l_dict = hist_of_image(source_image_l)

# source_image_l_eq, cdf_source  = hist_eq(source_image_l, l_dict)
# print(cdf_source)

# # print(source_image_l_eq.shape, a.shape, b.shape)
# source_image_eq = np.stack((source_image_l_eq, a, b), axis = -1)
# source_image_eq = np.array(source_image_eq, dtype=np.uint8)
# cv2.imwrite('source_img_eq.jpg', cv2.cvtColor(source_image_eq, cv2.COLOR_LAB2BGR))


In [400]:
target_image_gray = cv2.imread('target_image_2.jpg',0)
# target_image_gray = cv2.cvtColor(target_image, cv2.COLOR_BGR2GRAY)
# cv2.imwrite('target_img_gray.jpg', target_image_gray)

# target_gray_dict = hist_of_image(target_image_gray.copy())

# target_image_eq, cdf_target  = hist_eq(target_image_gray.copy(), target_gray_dict)
# cv2.imwrite('target_img_eq.jpg', target_image_eq)

In [401]:
target_image_gray_to_rgb = target_image_gray.copy()
target_image_gray_l = target_image_gray.copy()
# target_image_gray_to_rgb = cv2.cvtColor(target_image_gray_to_rgb, cv2.COLOR_GRAY2RGB)
# target_image_gray_to_lab = cv2.cvtColor(target_image_gray_to_rgb, cv2.COLOR_RGB2LAB)
# target_image_gray_l = target_image_gray_to_lab[:, :, 0]

In [116]:
# #source_image (LAB space), cdf_source, cdf_target, target_image_gray (GRAY scale space)

# cdf_target_inv = dict(map(reversed, cdf_target.items()))
# # print(cdf_target_inv)



# source_image_l = source_image[:, :, 0] #L channel of source image
# a, b = source_image[:, :, 1], source_image[:, :, 2]
# print(source_image_l.shape)
# for i in range(source_image_l.shape[0]):
#     for j in range(source_image_l.shape[1]):
#         min_dist = float('inf')
#         min_idx = -1
#         pixel_src = source_image_l[i,j]
#         prob_pixel_src = cdf_source[pixel_src]
#         for prob_pixel_target in cdf_target_inv.keys():
#             dist = np.abs(prob_pixel_target - prob_pixel_src)
#             if dist < min_dist:
#                 min_dist = dist
#                 min_idx = prob_pixel_target
#         source_image_l[i, j] = cdf_target_inv[min_idx]

# source_image_matched = np.stack((source_image_l, a, b), axis = -1)
# source_image_matched = np.array(source_image_matched, dtype=np.uint8)
# cv2.imwrite('source_img_matched.jpg', cv2.cvtColor(source_image_matched, cv2.COLOR_LAB2BGR))         
        


        

(480, 640)


True

In [118]:
# import cv2
# import numpy as np

# def histogram_match_lab(rgb_img, gray_img):
#     # Convert RGB image to LAB color space
#     lab_img = cv2.cvtColor(rgb_img, cv2.COLOR_RGB2LAB)

#     # Calculate histograms for the luminance channel of both images
#     hist_rgb, _ = np.histogram(lab_img[:,:,0], bins=256, range=(0, 256))
#     hist_gray, _ = np.histogram(gray_img, bins=256, range=(0, 256))

#     # Calculate cumulative distribution functions (CDFs)
#     cdf_rgb = hist_rgb.cumsum() / hist_rgb.sum()
#     cdf_gray = hist_gray.cumsum() / hist_gray.sum()

#     # Create a mapping from the RGB luminance values to grayscale CDF values
#     mapping = np.interp(cdf_rgb, cdf_gray, np.arange(256))

#     # Apply the mapping to the luminance channel of the LAB image
#     lab_img[:,:,0] = np.interp(lab_img[:,:,0], np.arange(256), mapping)

#     # Convert the matched LAB image back to RGB
#     matched_rgb_img = cv2.cvtColor(lab_img, cv2.COLOR_LAB2RGB)

#     return matched_rgb_img

# # Read the RGB and grayscale images
# rgb_image = cv2.imread('artistic_image.jpg')
# rgb_image = cv2.cvtColor(rgb_image, cv2.COLOR_BGR2RGB)
# gray_image = cv2.imread('target_img_gray.jpg', cv2.IMREAD_GRAYSCALE)

# # Perform histogram matching
# matched_rgb_image = histogram_match_lab(rgb_image, gray_image)

# # Display or save the matched RGB image
# cv2.imwrite('sample.jpg', cv2.cvtColor(matched_rgb_image, cv2.COLOR_RGB2BGR))


True

In [531]:
def histogram_matching(source_image, target_image):
    # Calculate histograms of the source and target images
    source_hist, _ = np.histogram(source_image.flatten(), bins=256, range=(0, 256), density=True)
    target_hist, _ = np.histogram(target_image.flatten(), bins=256, range=(0, 256), density=True)

    # Calculate cumulative distribution functions (CDF)
    source_cdf = source_hist.cumsum()
    target_cdf = target_hist.cumsum()

    # Create a mapping from source CDF to target CDF
    mapping = np.interp(source_cdf, target_cdf, np.arange(256))

    # Apply mapping to the source image
    matched_image = mapping[source_image]

    return matched_image

In [533]:
#3 Color Transfer

#Global transfer

# source_image = img.copy()
# cv2.imwrite('samplecheck1.jpg', cv2.cvtColor(source_image, cv2.COLOR_RGB2BGR))

def match_images(source_image, target_image):
    source_image_lab = cv2.cvtColor(source_image, cv2.COLOR_RGB2LAB)
    source_image_l_norm = source_image_lab[:, :, 0]
    a, b = source_image_lab[:, :, 1], source_image_lab[:, :, 2]
    source_image_l_norm = np.array(source_image_l_norm, dtype=np.float64) / 255.0
    # print("CHECK MAX AND MIN OF SOURCE: ", np.max(source_image_l_norm), np.min(source_image_l_norm))


    # target_image_wo_norm = target_image_gray.copy()
    target_image_wo_norm = target_image.copy()

    # print(target_image_wo_norm)
    # cv2.imwrite('samplecheck1.jpg', target_image_norm)
    target_image_norm = np.array(target_image_wo_norm, dtype=np.float64) / 255.0
    # print("CHECK MAX AND MIN OF TARGET: ", np.max(target_image_norm), np.min(target_image_norm))

    # print(target_image_norm)
    # print(source_image_l_norm)

    mu_target = np.mean(target_image_norm)
    sigma_target = np.std(target_image_norm)

    mu_source = np.mean(source_image_l_norm)
    sigma_source = np.std(source_image_l_norm)

    source_image_l_linshift_pre = ((sigma_target/sigma_source) * (source_image_l_norm - mu_source)) + mu_target

    source_image_l_linshift = (source_image_l_linshift_pre - np.min(source_image_l_linshift_pre)) / (np.max(source_image_l_linshift_pre) - np.min(source_image_l_linshift_pre))

    source_image_l_linshift *= 255.0
    source_image_l_linshift = np.clip(source_image_l_linshift, 0, 255)

    source_image_linshift = np.dstack((source_image_l_linshift, a, b))

    return source_image_linshift

source_img = cv2.imread('artistic_image.jpg')
source_img_rgb = cv2.cvtColor(source_img, cv2.COLOR_BGR2RGB)
target_img = cv2.imread('target_image_2.jpg', 0)

source_image_linshift = match_images(source_img_rgb, target_img)

# source_image_lab = cv2.cvtColor(source_img_rgb, cv2.COLOR_RGB2LAB)
# l = source_image_lab[:, :, 0]
# a, b = source_image_lab[:, :, 1], source_image_lab[:, :, 2]

# source_image_l = histogram_matching(l, target_img)

# source_image_linshift = np.dstack((source_image_l, a, b))

# source_image_l_linshift = np.clip(source_image_l_linshift, 0.0, 255.0)

source_image_linshift_uint = np.array(source_image_linshift, dtype=np.uint8)
# cv2.imwrite("l_linshift_check.jpg", np.array(source_image_l_linshift, dtype=np.uint8))


cv2.imwrite('source_image_linshift.jpg', cv2.cvtColor(source_image_linshift_uint, cv2.COLOR_LAB2BGR))
# print(source_image_l_linshift)


#test

# mu_target = np.mean(target_image_wo_norm)
# sigma_target = np.std(target_image_wo_norm)

# mu_source = np.mean(source_image_l_linshift)
# sigma_source = np.std(source_image_l_linshift)

# print(mu_target, sigma_target, mu_source, sigma_source)
# print()


True

In [515]:
# #Remove it at the end
# #histogram matching
# # source_image_linshift = source_image.copy()

# import cv2

# def match_histograms(source_gray, target_gray):
#     # Convert images to grayscale
#     # source_gray = cv2.cvtColor(source_image, cv2.COLOR_BGR2GRAY)
#     # target_gray = cv2.cvtColor(target_image, cv2.COLOR_BGR2GRAY)

#     # Calculate histograms of the grayscale images
#     source_hist, _ = np.histogram(source_gray.flatten(), bins=256, range=[0, 256])
#     target_hist, _ = np.histogram(target_gray.flatten(), bins=256, range=[0, 256])

#     # Calculate cumulative distribution functions (CDFs)
#     source_cdf = source_hist.cumsum() / source_hist.sum()
#     target_cdf = target_hist.cumsum() / target_hist.sum()

#     # Create a lookup table for histogram matching
#     lookup_table = np.interp(source_cdf, target_cdf, np.arange(256))

#     # Apply histogram matching to the source grayscale image
#     matched_gray = cv2.LUT(source_gray, lookup_table)

#     # Convert matched grayscale image to color (BGR) for visualization
#     #matched_color = cv2.cvtColor(matched_gray, cv2.COLOR_GRAY2BGR)

#     return matched_gray

# # Load your two grayscale images
# image1 = source_image_lab[:, :, 0].copy()
# a, b = source_image_lab[:, :, 1].copy(), source_image_lab[:, :, 2].copy()
# image2 = target_image_gray_l.copy()

# # Match the histogram of image1 to the levels of image2
# source_image_l_linshift = match_histograms(image1, image2)
# source_image_linshift = np.array(np.stack((source_image_l_linshift, a, b), axis=-1), dtype=np.uint8)

# cv2.imwrite('source_image_linshift.jpg', cv2.cvtColor(source_image_linshift, cv2.COLOR_LAB2BGR))


In [534]:
##jitter sampling

# import cv2
# import numpy as np
import random

locations = []

def jitter_sampling(image_lab, num_colors):
    h, w, _ = image_lab.shape
    image_lab = np.array(image_lab, dtype=np.float64)
    # print(image_lab.shape)
    l, a, b = image_lab[:, :, 0].copy(), image_lab[:, :, 1].copy(), image_lab[:, :, 2].copy()

    grid_size = int(np.ceil(np.sqrt(num_colors)))
    grid_height = h // grid_size
    grid_width = w // grid_size
    # print(grid_height, grid_width)

    sampled_colors = []

    for i in range(grid_size):
        for j in range(grid_size):
            top = i * grid_height
            bottom = (i + 1) * grid_height
            left = j * grid_width
            right = (j + 1) * grid_width

            rand_y = random.randint(top, bottom - 1)
            rand_x = random.randint(left, right - 1)

            locations.append([l[rand_y, rand_x], a[rand_y, rand_x], b[rand_y, rand_x], rand_y, rand_x])
            
            # print(rand_y, rand_x, image_lab[rand_y, rand_x])

            val = np.array([l[rand_y, rand_x], a[rand_y, rand_x], b[rand_y, rand_x], rand_y, rand_x], dtype=np.float64)
            sampled_colors.append(val)

    print(locations)
    
    if len(sampled_colors) > num_colors:
        sampled_colors = random.sample(sampled_colors, num_colors)

    return sampled_colors

image_path = source_image_linshift.copy()
num_colors = 200 #As mentioned in paper

# cv2.imwrite('test2.jpg', cv2.cvtColor(image_path, cv2.COLOR_LAB2BGR))

sampled_colors = jitter_sampling(image_path, num_colors)
sampled_colors = np.array(sampled_colors, dtype=np.float64)
# print(len(sampled_colors))
print(sampled_colors)

[[162.79578063805482, 130.0, 80.0, 8, 4], [163.81276719252952, 127.0, 83.0, 7, 63], [164.75319708541147, 126.0, 85.0, 8, 91], [164.75319708541147, 123.0, 91.0, 1, 133], [171.94063544318905, 120.0, 93.0, 29, 202], [176.11955650981525, 119.0, 97.0, 0, 230], [171.28948768698282, 121.0, 94.0, 12, 280], [176.63029373556583, 117.0, 100.0, 28, 318], [177.11124019440655, 117.0, 101.0, 5, 374], [182.3834132368519, 118.0, 107.0, 31, 395], [185.28825032622876, 117.0, 111.0, 31, 431], [186.0968724658248, 117.0, 114.0, 8, 502], [187.49706983728706, 118.0, 118.0, 11, 525], [188.62104681908227, 119.0, 120.0, 23, 550], [192.1005217292622, 116.0, 124.0, 22, 593], [168.24765332807388, 121.0, 90.0, 52, 17], [169.40664482919982, 121.0, 91.0, 50, 78], [173.46136171788865, 119.0, 96.0, 54, 121], [170.66094576365066, 120.0, 93.0, 32, 142], [176.63029373556583, 117.0, 100.0, 60, 175], [173.46136171788865, 119.0, 95.0, 41, 217], [177.76468174454297, 115.0, 102.0, 46, 257], [184.20314281235747, 112.0, 111.0, 52

In [513]:
print(image_path[186, 488])

[ 36.90789474 121.         143.        ]


In [535]:
def calc_distance_val(p1, p2):
    #print(abs(p1 - p2))
    return abs(float(p1) - float(p2))

def compute_sd(image, neighborhood_size=5):
    amt_to_pad = (neighborhood_size - 1) // 2
    y, x = image.shape
    sds = np.zeros((y, x))

    padded = np.pad(image, ((amt_to_pad, amt_to_pad), (amt_to_pad, amt_to_pad)), mode='edge')
    padded = np.array(padded, dtype=np.float64)

    for i in range(amt_to_pad, y + amt_to_pad):
        for j in range(amt_to_pad, x + amt_to_pad):
            region = padded[i - amt_to_pad:i + amt_to_pad + 1, j - amt_to_pad:j + amt_to_pad + 1]
            # sd = np.sqrt(np.mean(np.square(region - np.mean(region))))
            #sd = np.sqrt(region.var())
            sd = np.std(region)
            sds[i - amt_to_pad, j - amt_to_pad] = sd
    return sds

target_img = cv2.imread('target_image_2.jpg', 0)
sds = compute_sd(target_img, 5)
print(sds.shape)
print(sds)
print(np.min(sds), np.max(sds))

(471, 600)
[[1.05527248 0.90862534 0.8908423  ... 1.09544512 1.28747816 1.25602548]
 [1.20929732 1.0851728  1.05375519 ... 1.52105227 1.57784663 1.35410487]
 [1.21720992 1.09105454 1.02449988 ... 1.49666295 1.52315462 1.29614814]
 ...
 [3.73823488 3.58083789 3.37615166 ... 8.51765226 8.52065725 8.37768464]
 [3.08700502 3.03684046 3.03315018 ... 7.74741247 7.65913833 7.46190324]
 [2.09914268 2.36169431 2.54684118 ... 4.38068488 4.5491098  4.687686  ]]
0.44542114902640173 59.33007331868046


In [536]:
# #test
# lt = []
# h, w, _ = image_path.shape
# for i in range(h):
#     for j in range(w):
#         lt.append(image_path[i, j, 1])
# lt = np.array(lt)
# print(np.max(lt), np.min(lt))

In [537]:
def coloring(image, sds):
    # image = target_image.copy()

    # image = np.array(image, dtype=np.uint8)
    # colored_image_lab = np.zeros((image.shape[0], image.shape[1], 3), dtype=np.uint8)

    # cv2.imwrite('gradient/sample1.jpg', colored_image_lab)
    # cv2.imwrite('gradient/sample2.jpg', image)

    image = np.array(image, dtype=np.float64)
    colored_image_lab = np.zeros((image.shape[0], image.shape[1], 3), dtype=np.float64)

    for i in range(colored_image_lab.shape[0]):
        for j in range(colored_image_lab.shape[1]):
            sd = sds[i,j]
            weighted_lum = (0.9*image[i, j] + 0.1*sd)
            # weighted_lum = image[i, j]
            # if image[i, j] - sd > 30.0:
            #     print("W,S,I:", weighted_lum, sd, image[i, j])

            colored_image_lab[i, j, 0] = image[i, j]
            min_lum = np.inf
            for color in sampled_colors:
                lum = color[0]
                distance = calc_distance_val(lum, weighted_lum)

                if distance < min_lum:
                    min_lum = distance
                    # colored_image_lab[i, j, 0] = color[0]
                    #print('color: ', np.array([image[i - amt_to_pad, j - amt_to_pad], color[1], color[2]]))
                    colored_image_lab[i, j, 1], colored_image_lab[i, j, 2]  = color[1].copy(), color[2].copy()#image_path[int(color[3]), int(color[4])]
                else:
                    continue
            # print(colored_image_lab[i, j])
    return colored_image_lab

target_image_to_color = cv2.imread('target_image_2.jpg', 0)
colored_image_lab = coloring(target_image_to_color, sds)


In [528]:
# # print(np.max(colored_image_lab[:,:,1]), np.min(colored_image_lab[:,:,1]))

# def coloring(source_image, target_image, sds, sampled_colors, patch_size=5):
#     target_image = np.array(target_image, dtype=np.float64)
#     colored_image_lab = np.zeros((target_image.shape[0], target_image.shape[1], 3))

#     pad_amount = patch_size // 2
#     padded_image = np.pad(target_image.copy(), ((pad_amount, pad_amount), (pad_amount, pad_amount)), mode='edge')

#     padded_image_src = np.pad(source_image, ((pad_amount, pad_amount), (pad_amount, pad_amount)), mode='edge')
    
#     print(sds.shape)
#     for j in range(pad_amount, padded_image.shape[0]- pad_amount):
#         for i in range(pad_amount, padded_image.shape[1] - pad_amount):
#             #print(j - pad_amount, i - pad_amount)
#             sd = sds[j - pad_amount, i - pad_amount]
#             patch = padded_image[j-pad_amount:j+pad_amount+1, i-pad_amount:i+pad_amount+1]

#             # weighted_lum = (0.5 * error + 0.5 * sd)
            
#             min_distance = np.inf
#             best_color = None
            
#             for color in sampled_colors:
#                 loc_j, loc_i = int(color[3]), int(color[4])
#                 patch_src = padded_image_src[loc_j:loc_j+patch_size, loc_i:loc_i+patch_size]

#                 error = np.sum(np.abs(patch_src - patch))

#                 # weighted_lum = (0.5 * error + 0.5 * sd)
#                 weighted_lum = error
                
#                 if weighted_lum < min_distance:
#                     min_distance = weighted_lum
#                     best_color = color
            
#             if best_color is not None:
#                 # print(colored_image_lab[i, j])
#                 colored_image_lab[j-pad_amount, i-pad_amount] = np.array([target_image[j-pad_amount, i-pad_amount], best_color[1], best_color[2]])
                
#     return colored_image_lab

# target_image = cv2.imread('target_image_2.jpg', 0)
# # target_image = cv2.cvtColor(target_image, cv2.COLOR_BGR2LAB)
# target_image_to_color = target_image.copy()

# # cv2.imwrite('wo_swatch/target_image_gray.jpg', target_image_to_color)

# source_image = source_image_linshift.copy()
# source_image_l = source_image[:, :, 0]
# colored_image_lab = coloring(source_image_l, target_image_to_color, sds, sampled_colors, 5)

In [538]:
colored_image_lab_uint = np.array(colored_image_lab, dtype=np.uint8)

cv2.imwrite('gray_to_color_wo_swatch.jpg', cv2.cvtColor(colored_image_lab_uint, cv2.COLOR_LAB2BGR))
# cv2.imwrite('gray_to_color_wo_swatch.jpg', colored_image_lab)

True

In [501]:
#With Swatches
def jitter_sampling_swatch(img, image_lab, num_colors, y_add, x_add):
    h, w, _ = image_lab.shape
    # print(image_lab.shape)
    l, a, b = img[:, :, 0].copy(), img[:, :, 1].copy(), img[:, :, 2].copy()

    grid_size = int(np.ceil(np.sqrt(num_colors)))
    grid_height = h // grid_size
    grid_width = w // grid_size
    # print(grid_height, grid_width)

    sampled_colors = []
    locations = []

    for i in range(grid_size):
        for j in range(grid_size):
            top = i * grid_height
            bottom = (i + 1) * grid_height
            left = j * grid_width
            right = (j + 1) * grid_width

            rand_y = random.randint(top, bottom - 1)
            rand_x = random.randint(left, right - 1)

            locations.append([l[rand_y + y_add, rand_x + x_add], a[rand_y + y_add, rand_x + x_add], b[rand_y + y_add, rand_x + x_add], rand_y + y_add, rand_x + x_add])
            # print(rand_y, rand_x, image_lab[rand_y, rand_x])

            val = np.array([l[rand_y + y_add, rand_x + x_add], a[rand_y + y_add, rand_x + x_add], b[rand_y + y_add, rand_x + x_add], rand_y + y_add, rand_x + x_add], dtype=np.float64)
            sampled_colors.append(val)
    
    if len(sampled_colors) > num_colors:
        sampled_colors = random.sample(sampled_colors, num_colors)

    print(locations)
    return sampled_colors

In [502]:
# With Swatches

#[y1, x1, y2, x2] = [top, left, bottom, right] : sky, water, tree
source_swatches = [[35, 20, 115, 105], [393, 586, 439, 636], [150, 530, 222, 590], [175, 585, 212, 611], [10, 188, 49, 307]]#, [215, 170, 250, 225]]

def extract_colors(source_image, source_swatch, num_colors):
    # padding = window_size//2
    y1, x1, y2, x2 = source_swatch
    # padded = np.pad(source_image, ((padding, padding), (padding, padding)), mode='edge')
    # padded = np.array(padded, dtype=np.float64)
    swatch = source_image[y1:y2,x1:x2,:]
    colors = jitter_sampling_swatch(source_image, swatch, num_colors, y1, x1)
    return colors
    
# def color_transfer(source_swatch, target_swatch):

source_image_path = 'artistic_image.jpg'
source_image = cv2.imread(source_image_path)
source_image_lab = cv2.cvtColor(source_image, cv2.COLOR_BGR2LAB)
target_image = cv2.imread('target_image_2.jpg', 0)

list_colors_per_swatch = []
for swatch in source_swatches:
    list_colors_per_swatch.append(extract_colors(source_image_lab, swatch, 50))


[[166, 124, 88, 40, 27], [168, 122, 90, 42, 37], [168, 122, 89, 40, 48], [168, 122, 89, 44, 55], [168, 123, 88, 36, 69], [170, 121, 90, 43, 79], [171, 122, 90, 43, 85], [170, 121, 91, 43, 90], [169, 122, 90, 49, 21], [169, 122, 90, 46, 33], [170, 121, 90, 52, 41], [168, 122, 89, 45, 54], [170, 121, 90, 45, 69], [170, 121, 90, 47, 73], [169, 122, 90, 46, 88], [172, 120, 92, 47, 97], [171, 121, 91, 63, 20], [172, 121, 91, 61, 34], [172, 120, 91, 57, 45], [172, 121, 91, 58, 59], [171, 122, 90, 55, 63], [173, 120, 93, 56, 77], [174, 119, 94, 59, 85], [176, 119, 94, 61, 90], [173, 120, 94, 65, 26], [173, 120, 94, 74, 34], [176, 119, 95, 74, 43], [175, 120, 94, 68, 54], [175, 119, 95, 72, 61], [176, 119, 94, 67, 78], [176, 120, 94, 72, 82], [176, 119, 96, 66, 96], [176, 121, 94, 80, 29], [178, 119, 95, 78, 39], [179, 119, 96, 81, 43], [180, 118, 97, 84, 59], [180, 118, 97, 79, 65], [178, 118, 96, 77, 77], [180, 116, 97, 77, 87], [179, 119, 96, 82, 98], [180, 120, 97, 94, 28], [174, 121, 94, 

In [503]:
# list_colors_per_swatch_cons = []
# list_colors_per_swatch_cons.append(list_colors_per_swatch[0])
# list_colors_per_swatch_cons.append(list_colors_per_swatch[2])
# print(source_image_lab[153,547])


In [504]:
def compute_ssd_patch(source_patch, target_patch):
    return np.sum((source_patch - target_patch) ** 2)

def color_transfer(source_image, target_image, target_swatch, target_color_image, colors, window_size=5):
    padding = window_size//2

    padded_source = np.pad(source_image, ((padding, padding), (padding, padding)), mode='edge')
    padded_source = np.array(padded_source, dtype=np.float64)
    # print(padded_source.shape)

    padded_target = np.pad(target_image, ((padding, padding), (padding, padding)), mode='edge')
    padded_target = np.array(padded_target, dtype=np.float64)
    # print(padded_target.shape)

    target_image = np.array(target_image, dtype=np.float64)
    # print(target_image)
    # print(padded_target)
    # l, a, b = np.zeros((h, w)), np.zeros(h, w), np.zeros(h, w)

    y1, x1, y2, x2 = target_swatch
    y1 += padding
    x1 += padding
    y2 += padding
    x2 += padding

    for y in range(y1, y2):
        for x in range(x1, x2):
            target_patch = padded_target[y - padding:y + padding+1, x - padding:x + padding+1]
            min_ssd = np.inf
            #print(target_patch)

            best_match_color = None

            for color in colors:
                s_y, s_x = int(color[3]), int(color[4])
                s_y += padding
                s_x += padding
                source_patch = padded_source[s_y - padding:s_y + padding+1, s_x - padding:s_x + padding+1]

                ssd = compute_ssd_patch(source_patch, target_patch)
                if ssd < min_ssd:
                    min_ssd = ssd
                    best_match_color = color
            
            target_color_image[y-padding, x-padding] = np.array([target_image[y-padding, x-padding], best_match_color[1].copy(), best_match_color[2].copy()])
    return target_color_image


h, w = target_image.shape
l_ch = target_image.copy()
a_ch, b_ch = np.zeros((h,w)), np.zeros((h,w))
target_color_image = np.dstack((l_ch, a_ch, b_ch))
# target_color_image = np.zeros((h, w, 3))

target_swatches = [[150, 110, 210, 200], [355, 70, 405, 195], [263, 74, 305, 150], [314, 150, 323, 265], [25, 300, 65, 415]] 
# target_swatch = [115, 0, 168, 258]
# target_swatch = [0, 0, 168, 258]
# color = list_colors_per_swatch[2]

# color = []
# for sublist in list_colors_per_swatch_cons:
#     color.extend(sublist)

#source_image_lab[:, :, 0] processing to match target_gray image intensity
source_img = cv2.imread(source_image_path)
source_image_rgb = cv2.cvtColor(source_img, cv2.COLOR_BGR2RGB)
source_image_lab = match_images(source_image_rgb, l_ch)

source_image_lab_uint = np.array(source_image_lab, dtype=np.uint8)
cv2.imwrite('source_image_linshift_swatch.jpg', cv2.cvtColor(source_image_lab_uint, cv2.COLOR_LAB2BGR))

for color, target_swatch in zip(list_colors_per_swatch, target_swatches):

    target_color_image = color_transfer(source_image_lab[:, :, 0].copy(), l_ch, target_swatch, target_color_image, color)
    
target_color_image_save = np.array(target_color_image, dtype=np.uint8)

cv2.imwrite('target_color_image.jpg', cv2.cvtColor(target_color_image_save, cv2.COLOR_LAB2BGR))


True

In [505]:
# target_swatch = [65, 132, 90, 170]
# color = list_colors_per_swatch_cons[1]

# target_color_image = color_transfer(source_image_lab[:, :, 0], target_image, target_swatch, target_color_image, color)

# target_color_image_save = np.array(target_color_image, dtype=np.uint8)

# cv2.imwrite('target_color_image_stage2.jpg', cv2.cvtColor(target_color_image_save, cv2.COLOR_LAB2BGR))

In [506]:
src_swatches = [[150, 110, 210, 200], [355, 70, 405, 195], [263, 74, 305, 150], [314, 150, 323, 265], [25, 300, 65, 415]] 
src_colors = []

for swatch in src_swatches:
    color = extract_colors(target_color_image, swatch, 50)
    src_colors.extend(color)

[[166.0, 124.0, 88.0, 156, 119], [167.0, 124.0, 88.0, 150, 124], [165.0, 124.0, 88.0, 155, 142], [164.0, 124.0, 88.0, 150, 150], [166.0, 124.0, 88.0, 153, 160], [166.0, 124.0, 88.0, 155, 171], [165.0, 124.0, 88.0, 152, 185], [164.0, 124.0, 88.0, 151, 197], [169.0, 124.0, 88.0, 163, 117], [168.0, 124.0, 88.0, 161, 122], [167.0, 124.0, 88.0, 163, 139], [168.0, 124.0, 88.0, 161, 146], [170.0, 124.0, 88.0, 163, 160], [166.0, 124.0, 88.0, 158, 173], [169.0, 124.0, 88.0, 162, 178], [170.0, 124.0, 88.0, 161, 191], [170.0, 124.0, 88.0, 165, 117], [168.0, 124.0, 88.0, 170, 123], [172.0, 124.0, 88.0, 167, 137], [168.0, 124.0, 88.0, 165, 149], [171.0, 124.0, 88.0, 167, 162], [168.0, 124.0, 88.0, 164, 165], [170.0, 124.0, 88.0, 166, 186], [170.0, 124.0, 88.0, 166, 195], [173.0, 124.0, 88.0, 177, 110], [172.0, 124.0, 88.0, 174, 128], [170.0, 124.0, 88.0, 172, 142], [173.0, 124.0, 88.0, 174, 144], [172.0, 124.0, 88.0, 171, 156], [171.0, 124.0, 88.0, 174, 165], [172.0, 124.0, 88.0, 174, 177], [176.0,

In [507]:
# cv2.imwrite('target_sample.jpg', cv2.cvtColor(np.array(target_color_image, dtype=np.uint8), cv2.COLOR_LAB2BGR))

In [508]:
target_swatch = [0, 0, 471, 600]
target_color_image = color_transfer(target_color_image[:, :, 0], target_color_image[:, :, 0], target_swatch, target_color_image, src_colors)

target_color_image_save = np.array(target_color_image, dtype=np.uint8)

cv2.imwrite('target_color_image_stage3.jpg', cv2.cvtColor(target_color_image_save, cv2.COLOR_LAB2BGR))

True

In [539]:
img = cv2.imread('target_image_2.jpg', 0)
cv2.imwrite('target_image.jpg', img)

True

In [13]:
### Ablations ###


import cv2
import numpy as np

path = 'Experiment_1/'
path2 = 'ablation/'
print('')

img_bgr = cv2.imread(path + 'source.jpg') #Image is being read in BGR color palette
img = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
img_gray = cv2.imread(path + 'source.jpg', 0)


#1.1.1 Shadow-Map Generation

#RGB to HSI

img_np = np.asarray(img)

h, w, c = img_np.shape

I = np.zeros((h, w, 1))
V1 = np.zeros((h, w, 1))
V2 = np.zeros((h, w, 1))
S = np.zeros((h, w, 1))
H = np.zeros((h, w, 1))

hsi_mat = np.array([[1/3, 1/3, 1/3], [-(np.sqrt(6))/6, -(np.sqrt(6))/6, (np.sqrt(6))/3], [(1/np.sqrt(6)), -(2/np.sqrt(6)), 0]]) #Eqn 1

for y in range(h):
    for x in range(w):
        rgb = img_np[y][x]
        rgb = rgb.reshape((3, 1))
        out = np.matmul(hsi_mat, rgb)
        I[y][x] = out[0][0]
        V1[y][x] = out[1][0]
        V2[y][x] = out[2][0]
        S[y][x] = np.sqrt((np.square(V1[y][x])) + np.square(V2[y][x])) #Eqn 2
        if V1[y][x] != 1:
            if V1[y][x] == 0 and V2[y][x] == 0:
                H[y][x] = 0
                continue
            elif V1[y][x] == 0:
                H[y][x] = 0
                continue
            H[y][x] = np.arctan((V2[y][x])/V1[y][x]) #Eqn 3

H_norm = (H - np.min(H)) / (np.max(H) - np.min(H))
I_norm = (I - np.min(I)) / (np.max(I) - np.min(I))

#Computing r-map
r_map = (H_norm + 1) / (I_norm + 1) #Eqn 4

r_map_scaled = 255 * ((r_map - np.min(r_map)) / (np.max(r_map) - np.min(r_map)))


#Threshold for Shadow Map

P = {}

for y in range(r_map_scaled.shape[0]):
    for x in range(r_map_scaled.shape[1]):
        keys_list = P.keys()
        if int(r_map_scaled[y][x]) in keys_list:
            P[int(r_map_scaled[y][x])] += 1
        else:
            P[int(r_map_scaled[y][x])] = 1

for key in P.keys():
    P[key] = P[key] / (h * w)

T_dict = {}

for t in range(0, 256):
    W1, W2 = 0, 0
    for i in range(0, t+1):
        if i in P.keys():
            W1 += P[i]
    for i in range(t+1, 256):
        if i in P.keys():
            W2 += P[i]

    mu1, mu2 = 0, 0
    for i in range(0, t+1):
        if i in P.keys():
            mu1 += (i * P[i]) / W1
    for i in range(t+1, 256):
        if i in P.keys():
            mu2 += (i * P[i]) / W2

    t1, t2 = 0, 0
    for i in range(0, t+1):
        if i in P.keys():
            t1 += P[i] * ((i - mu1) ** 2)
    for i in range(t+1, 256):
        if i in P.keys():
            t2 += P[i] * ((i - mu2) ** 2)

    T_dict[t] = t1+t2


lt = []
for key in T_dict.keys():
    lt.append(T_dict[key])

T = lt.index(min(lt))


#Shadow map

s = np.where(r_map_scaled > T, 1, 0)
s_inv = np.where(r_map_scaled > T, 0, 1)
cv2.imwrite(path2 + f'shadow_map_{T}.jpg', s_inv * 255)

s = np.where(r_map_scaled > T-30, 1, 0)
s_inv_test = np.where(r_map_scaled > T-30, 0, 1)
cv2.imwrite(path2 + f'shadow_map_{T-30}.jpg', s_inv_test * 255)

s = np.where(r_map_scaled > T+30, 1, 0)
s_inv_test = np.where(r_map_scaled > T+30, 0, 1)
cv2.imwrite(path2 + f'shadow_map_{T+30}.jpg', s_inv_test * 255)

print('Shadow map is generated\n')


#Shadow Image

lmd_1 = 0.8
SI_1 = np.where(s_inv == 0, lmd_1 * img_bgr + (1 - lmd_1) * s_inv, img_bgr)

cv2.imwrite(path2 + f'shadow_image_lmb_{lmd_1}.jpg', SI_1)

lmd_1 = 0
SI_1_test = np.where(s_inv == 0, lmd_1 * img_bgr + (1 - lmd_1) * s_inv, img_bgr)

cv2.imwrite(path2 + f'shadow_image_lmb_{lmd_1}.jpg', SI_1_test)

lmd_1 = 0.99
SI_1_test = np.where(s_inv == 0, lmd_1 * img_bgr + (1 - lmd_1) * s_inv, img_bgr)

cv2.imwrite(path2 + f'shadow_image_lmb_{lmd_1}.jpg', SI_1_test)

lmd_2 = 1.3
SI_2 = np.where(s_inv == 0, lmd_2 * img_bgr + (1 - lmd_2) * s_inv, img_bgr)

cv2.imwrite(path2 + 'shadow_image_lmd_1.3.jpg', SI_2)

print(f'Shadow Image generated with lambda {lmd_1} and {lmd_2} is generated\n')



#1.1.2 Line Draft Generation


#Bilateral Filtering

def bilateral_filter(image, diameter, sigma_color, sigma_space):
    filtered_image = np.zeros_like(image)
    height, width = image.shape

    for y in range(height):
        for x in range(width):
            weighted_sum = 0
            normalization = 0

            for i in range(-diameter // 2, diameter // 2 + 1):
                for j in range(-diameter // 2, diameter // 2 + 1):
                    neighbor_y = y + i
                    neighbor_x = x + j

                    if 0 <= neighbor_y < height and 0 <= neighbor_x < width:
                        color_difference = float(image[neighbor_y, neighbor_x]) - float(image[y, x])

                        intensity_weight = np.exp(-np.square(color_difference) / (2 * sigma_color * sigma_color))
                        spatial_weight = np.exp(-(np.square(i) + np.square(j)) / (2 * sigma_space * sigma_space))

                        weight = intensity_weight * spatial_weight

                        weighted_sum += weight * image[neighbor_y, neighbor_x]
                        normalization += weight

            filtered_image[y, x] = weighted_sum / normalization

    return filtered_image


diameter = 5
sigma_colors = [50]
sigma_spaces = [50]
bil_img = None
for sigma_color in sigma_colors:
    for sigma_space in sigma_spaces:
        filtered_image = bilateral_filter(img_gray, diameter, sigma_color, sigma_space)
        filtered_image = np.array(filtered_image, dtype=np.uint8)
        if sigma_color == 50 and sigma_space == 50:
            bil_img = filtered_image
        cv2.imwrite(path2 + f'bilateral_filter_output_{diameter}_{sigma_color}_{sigma_space}.jpg', filtered_image)
        print('Bilateral Filter output is generated\n')

bil_opencv = cv2.bilateralFilter(img_gray.copy(), 7, 50.0, 50.0)
cv2.imwrite(path2 + 'bilateral_filter_output_opencv.jpg', bil_opencv)


# Edge Detection

#Best sigma_color, Best sigma_space 
bil_img = cv2.imread(path2 + 'bilateral_filter_output_5_50_50.jpg', 0)
print(bil_img.shape)

sobel_x = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
sobel_y = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]])

padded_image = np.pad(bil_img, ((1, 1), (1, 1)), mode='edge')

gradient_x = np.zeros_like(bil_img, dtype=np.float32)
gradient_y = np.zeros_like(bil_img, dtype=np.float32)

for y in range(bil_img.shape[0]):
    for x in range(bil_img.shape[1]):
        gradient_x[y, x] = np.sum(padded_image[y:y+3, x:x+3] * sobel_x)
        gradient_y[y, x] = np.sum(padded_image[y:y+3, x:x+3] * sobel_y)

edge_map = np.sqrt(gradient_x**2 + gradient_y**2)

cv2.imwrite(path2 + 'edge_map.jpg', edge_map)
print('Edge map output is generated\n')


#Line Draft using Threshold

ld_thresh = 130

line_draft = np.where(edge_map >= ld_thresh, 1, 0)

line_draft_inv = 255 * np.where(edge_map >= ld_thresh, 0, 1)

cv2.imwrite(path2 + 'line_draft_inv.jpg', line_draft_inv)
print('Line draft image is generated\n')


ld_thresh = 100

line_draft_test = np.where(edge_map >= ld_thresh, 1, 0)

line_draft_inv_test = 255 * np.where(edge_map >= ld_thresh, 0, 1)

cv2.imwrite(path2 + f'line_draft_inv_{ld_thresh}.jpg', line_draft_inv_test)
print('Line draft image is generated\n')

ld_thresh = 170

line_draft_test = np.where(edge_map >= ld_thresh, 1, 0)

line_draft_inv_test = 255 * np.where(edge_map >= ld_thresh, 0, 1)

cv2.imwrite(path2 + f'line_draft_inv_{ld_thresh}.jpg', line_draft_inv_test)
print('Line draft image is generated\n')


#1.2 Color Adjustment Step

lab_img = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2LAB)

l, a, b = lab_img[:,:,0], lab_img[:,:,1], lab_img[:,:,2]
l_mid = np.int8(60)

neutral_l = np.ones((h, w), dtype=np.uint8) * l_mid

chromatic_map = np.stack((neutral_l, a, b), axis = -1)

chromatic_map_rgb = cv2.cvtColor(chromatic_map, cv2.COLOR_LAB2RGB)

cv2.imwrite(path + 'chromatic_map.jpg', cv2.cvtColor(chromatic_map, cv2.COLOR_LAB2BGR))
print('Chromatic map is generated\n')


#SI in BGR

SI = SI_1.copy()
rho = 0.01 #[0.005, 0.2]

SI = np.array(SI, dtype=np.uint8)

SI = cv2.cvtColor(SI, cv2.COLOR_BGR2RGB)

SI_prime = SI * ((1 + np.tanh(rho * (chromatic_map_rgb - 128))) / 2)
SI_prime = np.array(SI_prime, dtype=np.uint8)

cv2.imwrite(path2 + 'shadow_image_color_enhanced.jpg', cv2.cvtColor(SI_prime, cv2.COLOR_RGB2BGR))
print('Shadow image is generated\n')


SI = SI_1.copy()
rho = 0.005 #[0.005, 0.2]

SI = np.array(SI, dtype=np.uint8)

SI = cv2.cvtColor(SI, cv2.COLOR_BGR2RGB)

SI_prime_test = SI * ((1 + np.tanh(rho * (chromatic_map_rgb - 128))) / 2)
SI_prime_test = np.array(SI_prime_test, dtype=np.uint8)

cv2.imwrite(path2 + f'shadow_image_color_enhanced_{rho}.jpg', cv2.cvtColor(SI_prime_test, cv2.COLOR_RGB2BGR))
print('Shadow image is generated\n')

SI = SI_1.copy()
rho = 0.2 #[0.005, 0.2]

SI = np.array(SI, dtype=np.uint8)

SI = cv2.cvtColor(SI, cv2.COLOR_BGR2RGB)

SI_prime_test = SI * ((1 + np.tanh(rho * (chromatic_map_rgb - 128))) / 2)
SI_prime_test = np.array(SI_prime_test, dtype=np.uint8)

cv2.imwrite(path2 + f'shadow_image_color_enhanced_{rho}.jpg', cv2.cvtColor(SI_prime_test, cv2.COLOR_RGB2BGR))
print('Shadow image is generated\n')


#Saturation Correction

saturation_scale = 1.3

SI_prime_HSV = cv2.cvtColor(SI_prime, cv2.COLOR_RGB2HSV)

H = SI_prime_HSV[:, :, 0]
S = SI_prime_HSV[:, :, 1]
V = SI_prime_HSV[:, :, 2]

S_corrected = np.array(np.round(255 * ((S - np.min(S)) / (np.max(S) - np.min(S))))).astype(int)

S_corrected = S_corrected * saturation_scale
S_corrected = np.clip(S_corrected, 0, 255)

S_corrected = np.array(S_corrected, dtype=np.uint8)

SI_corrected = np.stack((H, S_corrected, V), axis = -1) #Why not S

SI_corrected_RGB = cv2.cvtColor(SI_corrected, cv2.COLOR_HSV2RGB)

cv2.imwrite(path2 + 'shadow_image_corrected.jpg', cv2.cvtColor(SI_corrected, cv2.COLOR_HSV2BGR))
print('Shadow image with corrected saturation is generated\n')


saturation_scale = 1

SI_prime_HSV = cv2.cvtColor(SI_prime, cv2.COLOR_RGB2HSV)

H = SI_prime_HSV[:, :, 0]
S = SI_prime_HSV[:, :, 1]
V = SI_prime_HSV[:, :, 2]

S_corrected = np.array(np.round(255 * ((S - np.min(S)) / (np.max(S) - np.min(S))))).astype(int)

S_corrected = S_corrected * saturation_scale
S_corrected = np.clip(S_corrected, 0, 255)

S_corrected = np.array(S_corrected, dtype=np.uint8)

SI_corrected = np.stack((H, S_corrected, V), axis = -1) #Why not S

cv2.imwrite(path2 + f'shadow_image_corrected_{saturation_scale}.jpg', cv2.cvtColor(SI_corrected, cv2.COLOR_HSV2BGR))
print('Shadow image with corrected saturation is generated\n')

saturation_scale = 2

SI_prime_HSV = cv2.cvtColor(SI_prime, cv2.COLOR_RGB2HSV)

H = SI_prime_HSV[:, :, 0]
S = SI_prime_HSV[:, :, 1]
V = SI_prime_HSV[:, :, 2]

S_corrected = np.array(np.round(255 * ((S - np.min(S)) / (np.max(S) - np.min(S))))).astype(int)

S_corrected = S_corrected * saturation_scale
S_corrected = np.clip(S_corrected, 0, 255)

S_corrected = np.array(S_corrected, dtype=np.uint8)

SI_corrected = np.stack((H, S_corrected, V), axis = -1) #Why not S

cv2.imwrite(path2 + f'shadow_image_corrected_{saturation_scale}.jpg', cv2.cvtColor(SI_corrected, cv2.COLOR_HSV2BGR))
print('Shadow image with corrected saturation is generated\n')


#Artistic Enhanced Image
beta = 0.7 #[0, 1]

line_draft_inv_3d = np.stack([line_draft_inv, line_draft_inv, line_draft_inv], axis = -1) 

artistic_image = np.where(line_draft_inv_3d == 0, beta * SI_corrected_RGB, SI_corrected_RGB)

artistic_image = np.array(artistic_image, dtype=np.uint8)

cv2.imwrite(path2 + 'artistic_image.jpg', cv2.cvtColor(artistic_image, cv2.COLOR_RGB2BGR))
print('Final artistic image is generated\n')

beta = 0 #[0, 1]

line_draft_inv_3d = np.stack([line_draft_inv, line_draft_inv, line_draft_inv], axis = -1) 

artistic_image = np.where(line_draft_inv_3d == 0, beta * SI_corrected_RGB, SI_corrected_RGB)

artistic_image = np.array(artistic_image, dtype=np.uint8)

cv2.imwrite(path2 + f'artistic_image_{beta}.jpg', cv2.cvtColor(artistic_image, cv2.COLOR_RGB2BGR))
print('Final artistic image is generated\n')


beta = 1 #[0, 1]

line_draft_inv_3d = np.stack([line_draft_inv, line_draft_inv, line_draft_inv], axis = -1) 

artistic_image = np.where(line_draft_inv_3d == 0, beta * SI_corrected_RGB, SI_corrected_RGB)

artistic_image = np.array(artistic_image, dtype=np.uint8)

cv2.imwrite(path2 + f'artistic_image_{beta}.jpg', cv2.cvtColor(artistic_image, cv2.COLOR_RGB2BGR))
print('Final artistic image is generated\n')


Shadow map is generated

Shadow Image generated with lambda 0.99 and 1.3 is generated

Bilateral Filter output is generated

(438, 640)
Edge map output is generated

Line draft image is generated

Line draft image is generated

Line draft image is generated

Chromatic map is generated

Shadow image is generated

Shadow image is generated

Shadow image is generated

Shadow image with corrected saturation is generated

Shadow image with corrected saturation is generated

Shadow image with corrected saturation is generated

Final artistic image is generated

Final artistic image is generated

Final artistic image is generated



In [8]:
import cv2
import numpy as np
import random

image_path = input('Give folder name containing the source image and target image: ')

path = str(image_path) + '/'
print('')

def compute_ssd_patch(source_patch, target_patch):
    return np.sum((source_patch - target_patch) ** 2)

def are_images_equal(image1, image2):
    if image1.shape != image2.shape:
        raise ValueError("Images must have the same shape for MSE calculation.")

    squared_diff = np.square(image1 - image2)
    mse = np.mean(squared_diff)
    return mse

#3 Color Transfer

#Global transfer

def linshift_match_images(source_image, target_image):
    source_image_lab = cv2.cvtColor(source_image, cv2.COLOR_RGB2LAB)
    source_image_l_norm = source_image_lab[:, :, 0]
    a, b = source_image_lab[:, :, 1], source_image_lab[:, :, 2]
    source_image_l_norm = np.array(source_image_l_norm, dtype=np.float64) / 255.0

    target_image_wo_norm = target_image.copy()

    target_image_norm = np.array(target_image_wo_norm, dtype=np.float64) / 255.0

    mu_target = np.mean(target_image_norm)
    sigma_target = np.std(target_image_norm)

    mu_source = np.mean(source_image_l_norm)
    sigma_source = np.std(source_image_l_norm)

    source_image_l_linshift_pre = ((sigma_target/sigma_source) * (source_image_l_norm - mu_source)) + mu_target

    source_image_l_linshift = (source_image_l_linshift_pre - np.min(source_image_l_linshift_pre)) / (np.max(source_image_l_linshift_pre) - np.min(source_image_l_linshift_pre))

    source_image_l_linshift *= 255.0
    source_image_l_linshift = np.clip(source_image_l_linshift, 0, 255)

    source_image_linshift = np.dstack((source_image_l_linshift, a, b))

    return source_image_linshift

source_img = cv2.imread(path + 'artistic_image.jpg')
target_img_bgr = cv2.imread(path + 'target_image2.jpg')
target_img_lab = cv2.cvtColor(target_img_bgr, cv2.COLOR_BGR2LAB)
target_img = target_img_lab[:, :, 0]

source_img_rgb = cv2.cvtColor(source_img, cv2.COLOR_BGR2RGB)

source_image_linshift = linshift_match_images(source_img_rgb, target_img)

source_image_linshift_uint = np.array(source_image_linshift, dtype=np.uint8)

#cv2.imwrite('source_image_linshift.jpg', cv2.cvtColor(source_image_linshift_uint, cv2.COLOR_LAB2BGR))


#jitter sampling

def jitter_sampling(image_lab, num_colors):
    h, w, _ = image_lab.shape
    image_lab = np.array(image_lab, dtype=np.float64)

    l, a, b = image_lab[:, :, 0].copy(), image_lab[:, :, 1].copy(), image_lab[:, :, 2].copy()

    grid_size = int(np.ceil(np.sqrt(num_colors)))
    grid_height = h // grid_size
    grid_width = w // grid_size

    sampled_colors = []

    for i in range(grid_size):
        for j in range(grid_size):
            top = i * grid_height
            bottom = (i + 1) * grid_height
            left = j * grid_width
            right = (j + 1) * grid_width

            rand_y = random.randint(top, bottom - 1)
            rand_x = random.randint(left, right - 1)

            val = np.array([l[rand_y, rand_x], a[rand_y, rand_x], b[rand_y, rand_x], rand_y, rand_x], dtype=np.float64)
            sampled_colors.append(val)
    
    if len(sampled_colors) > num_colors:
        sampled_colors = random.sample(sampled_colors, num_colors)

    return sampled_colors

image_path = source_image_linshift.copy()
num_colors = 200 #As mentioned in paper

sampled_colors = jitter_sampling(image_path, num_colors)
sampled_colors = np.array(sampled_colors, dtype=np.float64)


def calc_distance_val(p1, p2):
    return abs(float(p1) - float(p2))


def compute_sd(image, neighborhood_size=5):
    amt_to_pad = (neighborhood_size - 1) // 2
    y, x = image.shape
    sds = np.zeros((y, x))

    padded = np.pad(image, ((amt_to_pad, amt_to_pad), (amt_to_pad, amt_to_pad)), mode='edge')
    padded = np.array(padded, dtype=np.float64)

    for i in range(amt_to_pad, y + amt_to_pad):
        for j in range(amt_to_pad, x + amt_to_pad):
            region = padded[i - amt_to_pad:i + amt_to_pad + 1, j - amt_to_pad:j + amt_to_pad + 1]
            sd = np.std(region)
            sds[i - amt_to_pad, j - amt_to_pad] = sd
    return sds

# target_img = cv2.imread(path + 'target_image2.jpg', 0)
sds = compute_sd(target_img, 5)


# def coloring(image, sds):
#     image = np.array(image, dtype=np.float64)
#     colored_image_lab = np.zeros((image.shape[0], image.shape[1], 3), dtype=np.float64)

#     for i in range(colored_image_lab.shape[0]):
#         for j in range(colored_image_lab.shape[1]):
#             sd = sds[i,j]
#             weighted_lum = (0.9*image[i, j] + 0.1*sd)

#             colored_image_lab[i, j, 0] = image[i, j]
#             min_lum = np.inf
#             for color in sampled_colors:
#                 lum = color[0]
#                 distance = calc_distance_val(lum, weighted_lum)

#                 if distance < min_lum:
#                     min_lum = distance
#                     colored_image_lab[i, j, 1], colored_image_lab[i, j, 2]  = color[1].copy(), color[2].copy()
#                 else:
#                     continue
#     return colored_image_lab

def coloring(source_image, target_image, target_swatch, target_color_image, colors, sds, window_size=5):
    padding = window_size//2
    print(padding)

    padded_source = np.pad(source_image, ((padding, padding), (padding, padding)), mode='edge')
    padded_source = np.array(padded_source, dtype=np.float64)

    padded_target = np.pad(target_image, ((padding, padding), (padding, padding)), mode='edge')
    padded_target = np.array(padded_target, dtype=np.float64)

    # target_image = np.array(target_image, dtype=np.float64)

    y1, x1, y2, x2 = target_swatch
    y1 += padding
    x1 += padding
    y2 += padding
    x2 += padding

    source_patches = []

    for color in colors:
        s_y, s_x = int(color[3]), int(color[4])
        s_y += padding
        s_x += padding
        source_patch = padded_source[s_y - padding:s_y + padding+1, s_x - padding:s_x + padding+1]
        source_patches.append(source_patch)
    source_patches = np.array(source_patches)

    for y in range(y1, y2):
        for x in range(x1, x2):
            target_patch = padded_target[y - padding:y + padding+1, x - padding:x + padding+1]
            min_ssd = np.inf

            best_match_color = None

            for color in range(len(colors)):
                source_patch = source_patches[color]

                ssd = compute_ssd_patch(source_patch, target_patch)
                if ssd < min_ssd:
                    min_ssd = ssd
                    best_match_color = colors[color]
            
            target_color_image[y-padding, x-padding, 1], target_color_image[y-padding, x-padding, 2]  = best_match_color[1].copy(), best_match_color[2].copy()
    return target_color_image

target_image_bgr = cv2.imread(path + 'target_image2.jpg')
target_img = cv2.cvtColor(target_image_bgr, cv2.COLOR_BGR2LAB)
targte_image = target_img[:, :, 0]

h, w = target_image.shape[0], target_image.shape[1]
a, b = np.zeros((h, w)), np.zeros((h, w))

target_swatch = [0, 0, h, w]
target_image_to_color = np.dstack((target_image.copy(), a, b))
colored_image_lab = coloring(source_image_linshift[:, :, 0], target_image.copy(), target_swatch, target_image_to_color, sampled_colors, sds, 5)

colored_image_lab_uint = np.array(colored_image_lab, dtype=np.uint8)

cv2.imwrite(path + 'colored_image_wo_swatch.jpg', cv2.cvtColor(colored_image_lab_uint, cv2.COLOR_LAB2BGR))
cv2.imwrite('colored_image_wo_swatch.jpg', cv2.cvtColor(colored_image_lab_uint, cv2.COLOR_LAB2BGR))
print('Target image with colors and without swatches generated\n')


2
Target image with colors and without swatches generated

